In [1]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

import dataloader
import layers

## The FER Dataset (Train / Dev / Test sets)

In [2]:
train, dev, test = dataloader.train_dev_test_sets()

In [3]:
print(f'Train set size: {len(train[0])}')
print(f'Dev set size: {len(dev[0])}')
print(f'Test set size: {len(test[0])}')
print()

print(f'Image shape: {dataloader.IMG_SHAPE}')
print(f'Number of classes: {len(dataloader.LABELS_MAP.values())}')
print(f'Classes: {list(dataloader.LABELS_MAP.values())}')
print()

Train set size: 28709
Dev set size: 3589
Test set size: 3589

Image shape: (48, 48, 1)
Number of classes: 7
Classes: ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']



## Create Placeholders

In [4]:
def create_placeholders(h, w, in_C, n_classes):
    with tf.name_scope('placeholders'):
        X = tf.placeholder(tf.float32, shape=[None, h, w, in_C], name='X')
        Y = tf.placeholder(tf.float32, shape=[None, n_classes], name='Y')
        
    return X, Y

## Forward-propagation

In [5]:
def forward_prop(X):
    # Conv2d: 8 filters, 7x7, strides=1, padding='SAME':
    X = layers.conv2d(X, n_filters=8, size=[7, 7], strides=[1, 1, 1, 1], padding='VALID', name='conv2d-layer-1')
    
    # MaxPool: 2x2, strides=1, padding='VALID'
    X = layers.maxpool(X, size=[2, 2], strides=[1, 1, 1, 1], padding='VALID', name='maxpool-layer-1')
    
    # ReLU
    X = tf.nn.relu(X, name='relu-layer-1')
    
    # Conv2d: 16 filters, 9x9, strides=1, padding='VALID':
    X = layers.conv2d(X, n_filters=16, size=[9, 9], strides=[1, 1, 1, 1], padding='VALID', name='conv2d-layer-2')
    
    # MaxPool: 2x2, strides=1, padding='VALID'
    X = layers.maxpool(X, size=[2, 2], strides=[1, 1, 1, 1], padding='VALID', name='maxpool-layer-2')
    
    # ReLU
    X = tf.nn.relu(X, name='relu-layer-1')
    
    # Flatten
    X = tf.layers.flatten(X, name='flatten')
    
    # FC 1
    X = layers.fully_connected(X, units=20, activation=tf.nn.sigmoid, name='fc-layer-1')
    
    # FC 2
    Z = layers.fully_connected(X, units=7, activation=None, name='fc-layers-2')
    
    return Z

## Compute Cost

In [6]:
def compute_cost(Y, Z):
    return tf.reduce_mean(
        tf.losses.softmax_cross_entropy(onehot_labels=Y, logits=Z)
    )

# Model

In [7]:
def model(train, dev, test, lr=1e-3, batch_size=32, num_epochs=5, run=1):
    tf.reset_default_graph()
    
    m, h, w, in_C = train[0].shape
    _, n_classes = train[1].shape
    num_batches_per_epoch = m // batch_size
    
    # Placeholders
    X, Y = create_placeholders(h, w, in_C, n_classes)
    
    # Forward-propagation
    Z = forward_prop(X)
    
    # Cost
    with tf.name_scope('xent'):
        cost = compute_cost(Y, Z)
        tf.summary.scalar('xent', cost)
        
    # Optimizer
    with tf.name_scope('optimizer'):
        optimize = tf.train.AdamOptimizer(lr).minimize(cost)
        
    # Prepare training data
    with tf.name_scope('train_dataset'):
        train_dataset = tf.data.Dataset.from_tensor_slices(train)
        train_dataset = train_dataset.repeat(num_epochs).shuffle(m*num_epochs).batch(batch_size)

        next_minibatch = train_dataset.make_one_shot_iterator().get_next()
        
    # Accuracy
    with tf.name_scope('accuracy'):
        predict_op = tf.argmax(Z, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
        tf.summary.scalar('accuracy', accuracy)
        
    summaries = tf.summary.merge_all()
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        k = 0
        epoch = 1
        epoch_cost = 0.
        epoch_acc = 0.
        writer = tf.summary.FileWriter('./boards/'+str(run))
        writer.add_graph(sess.graph)
        
        while True:
            try:
                minibatch_X, minibatch_Y = sess.run(next_minibatch)
                (_, tmp_cost, tmp_acc, s) = sess.run(
                    (optimize, cost, accuracy, summaries),
                    feed_dict={
                        X: minibatch_X,
                        Y: minibatch_Y
                    }
                )
                
                epoch_cost += tmp_cost / num_batches_per_epoch
                epoch_acc += tmp_acc / num_batches_per_epoch
            
                k+=1
                writer.add_summary(s, k)

            except tf.errors.OutOfRangeError:
                break
            
            if k % num_batches_per_epoch == 0:
                print(f'EPOCH {epoch}   |||   COST: {epoch_cost}   |||   Acurracy: {epoch_acc}')                
                epoch += 1
                epoch_cost = 0.
                epoch_acc = 0.

        train_accuracy = accuracy.eval({X: train[0], Y: train[1]})
        dev_accuracy = accuracy.eval({X: dev[0], Y: dev[1]})
        print('Train Accuracy:', train_accuracy)
        print('Dev Accuracy:', dev_accuracy)

In [9]:
model(train, dev, test, lr=1e-5, batch_size=32, num_epochs=10, run=1)

EPOCH 1   |||   COST: 1.9918139738382699   |||   Acurracy: 0.23784141583054594
EPOCH 2   |||   COST: 1.9239444868221718   |||   Acurracy: 0.2591973244147154
EPOCH 3   |||   COST: 1.8947951328528494   |||   Acurracy: 0.2654333890746935
EPOCH 4   |||   COST: 1.8681356365997006   |||   Acurracy: 0.2826435340022302
EPOCH 5   |||   COST: 1.8304020699317118   |||   Acurracy: 0.3040691192865118
EPOCH 6   |||   COST: 1.813671268082515   |||   Acurracy: 0.30831939799331226
EPOCH 7   |||   COST: 1.7835974795629626   |||   Acurracy: 0.3224986064659996
EPOCH 8   |||   COST: 1.7683098641792134   |||   Acurracy: 0.33329849498327924
EPOCH 9   |||   COST: 1.748855076771784   |||   Acurracy: 0.33761845039019117
EPOCH 10   |||   COST: 1.7283334401140236   |||   Acurracy: 0.33848940914158504
Train Accuracy: 0.34637222


NameError: name 'test_accuracy' is not defined